In [1]:
import re
import tarfile
import functools

import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

import spacy
nlp = spacy.load('en')

Using TensorFlow backend.


In [2]:
def tokenize(sent):
    sent = nlp(sent)
    token_list = []
    POV = []
    for token in sent:
        token_list.append(token.orth_)
        POV.append(token.pos_)
    print((token_list, POV))
tokenize(nlp('I love you.'))

(['I', 'love', 'you', '.'], ['PRON', 'VERB', 'PRON', 'PUNCT'])


In [3]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [4]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: functools.reduce(lambda x, y: x + y, data)
    story_position = tokenize(flatten(story))[1]
    data = [(flatten(story), flatten(story_position), q, q_position, answer) 
            for story, story_position, q, q_position, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [5]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xspos = []
    xqs = []
    xqspos = []
    ys = []
    for story, story_position, query, query_position, answer in data:
        x = [word_idx[w] for w in story]
        xpos = tokenize(x)[1]
        xq = [word_idx[w] for w in query]
        xqpos = tokenize(x)[1]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xspos.append(xpos)
        xqs.append(xq)
        xqspos.append(xqpos)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequence(xspos, maxlen=story_maxlen),
    pad_sequences(xqs, maxlen=query_maxlen), pad_sequence(xqspos, maxlen=query_maxlen), np.array(ys)

In [6]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [7]:
# Default QA1 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

AttributeError: 'str' object has no attribute 'orth_'

In [ ]:
vocab = set()
for story, story_position, q, q_position, answer in train + test:
    vocab |= set(story + story_position + q + q_position + [answer])
vocab = sorted(vocab)

In [ ]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

In [ ]:
print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

In [8]:
print('Build model...')

sentence = layers.Input(shape=(story[0]_maxlen,), dtype='int32')
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
encoded_sentence = layers.Dropout(0.3)(encoded_sentence)

sentence_position = layers.Input(shape=(story[1]_maxlen,), dtype='int32')
encoded_sentence_position = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence_position)
encoded_sentence_position = layers.Dropout(0.3)(encoded_sentence)

question = layers.Input(shape=(query[0]_maxlen,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question = layers.Dropout(0.3)(encoded_question)
encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question)
encoded_question = layers.RepeatVector(story_maxlen)(encoded_question)

question_position = layers.Input(shape=(query[1]_maxlen,), dtype='int32')
encoded_question_position = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question_position = layers.Dropout(0.3)(encoded_question)

merged = layers.add([encoded_sentence, encoded_sentence_position, encoded_question, encoded_question_position])
merged = RNN(EMBED_HIDDEN_SIZE)(merged)
merged = layers.Dropout(0.3)(merged)
preds = layers.Dense(vocab_size, activation='softmax')(merged)

model = Model([sentence, sentence_position, question, question_position], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

SyntaxError: invalid syntax (<ipython-input-8-28512f248d0d>, line 3)